In [9]:
import qiskit
from typing import List, Optional
from qiskit import QuantumCircuit
from qiskit.circuit import ParameterVector
from qiskit.exceptions import QiskitError

class AngleEmbedding:
    """Data embedding ins
    Class that implements layers of RX, RY, and/or RZ gates to embed data as quantum gate 
    parameters to be used for quantum machine learning applications.
    """
    def __init__(
        self,
        gate_sequence: List[str],
        num_qubits: int,
        repeat_parameters: Optional[bool] = False
    ):
        
        r"""
        Args:
            gate_sequence: Describes the sequence of gates applied. If multiple gates are applied,
            they will appear in the quantum circuit in they order they were inputted into the class
            as a list.
            
            Example:
            
            Below is the result of the sequence of gates ['RY', 'RX', 'RZ'] as the gate_sequence parameter.
            
            q_0: ┤ Ry(x[0]) ├┤ Rz(x[2]) ├┤ Rx(x[4]) ├
                 ├──────────┤├──────────┤├──────────┤
            q_1: ┤ Ry(x[1]) ├┤ Rz(x[3]) ├┤ Rx(x[5]) ├

            num_qubits: Integer that describes the number of qubits that angle embedding is applied on. Varies because the
            number of parameters possible to input into the circuit varies baseds on repeat_parameters.
            
            repeat_parameters: Default is false. Determines if quantum gate parameters can be repeated or not.
            
            Example:
            
            Below is the result of a circuit that has parameters repeated (repeat_parameters = True):
            
            q_0: ┤ Ry(x[0]) ├┤ Rz(x[0]) ├┤ Rx(x[0]) ├
                 ├──────────┤├──────────┤├──────────┤
            q_1: ┤ Ry(x[1]) ├┤ Rz(x[1]) ├┤ Rx(x[1]) ├
            
            Below is the result of a circuit that has parameters repeated (repeat_parameters = False), allowing for more 
            elements to be inputted into the embedding:

            q_0: ┤ Ry(x[0]) ├┤ Rz(x[2]) ├┤ Rx(x[4]) ├
                 ├──────────┤├──────────┤├──────────┤
            q_1: ┤ Ry(x[1]) ├┤ Rz(x[3]) ├┤ Rx(x[5]) ├
            
        **References:**
        [1] https://docs.pennylane.ai/en/stable/introduction/templates.html

        """
        
        if not isinstance(gate_sequence, list) or not all(isinstance(gate, str) for gate in gate_sequence):
            raise QiskitError("gate_sequence must be a list of strings.")
        
        if num_qubits <= 0:
            raise QiskitError("num_qubits must be a positive integer.")
        
        valid_gates = {'RY', 'RZ', 'RX'}
        if any(gate not in valid_gates for gate in gate_sequence):
            raise QiskitError(f"gate_sequence contains invalid gates. Valid gates are {valid_gates}.")
        
        self.gate_sequence = gate_sequence
        self.num_qubits = num_qubits
        self.repeat_parameters = repeat_parameters
    
    def _layer_sequence(self):
        layers = len(self.gate_sequence)
        x_length = self.num_qubits if self.repeat_parameters else self.num_qubits * layers
        x = ParameterVector('x', length=x_length)
        circuit = QuantumCircuit(self.num_qubits)
    
        index = 0
        for gate in self.gate_sequence:
            for j in range(self.num_qubits):
                param = x[j] if self.repeat_parameters else x[index]
                if gate == 'RY':
                    circuit.ry(param, j)
                elif gate == 'RZ':
                    circuit.rz(param, j)
                elif gate == 'RX':
                    circuit.rx(param, j)
                if not self.repeat_parameters:
                    index += 1
    
        return circuit

In [13]:
embedding = AngleEmbedding(['RY', 'RZ', 'RX'], 2, repeat_parameters=False)
circuit = embedding._layer_sequence()

In [14]:
circuit.draw()

┌──────────┐┌──────────┐┌──────────┐
q_0: ┤ Ry(x[0]) ├┤ Rz(x[2]) ├┤ Rx(x[4]) ├
     ├──────────┤├──────────┤├──────────┤
q_1: ┤ Ry(x[1]) ├┤ Rz(x[3]) ├┤ Rx(x[5]) ├
     └──────────┘└──────────┘└──────────┘